# Homework 6: Percentiles, Bootstrap, A/B Testing

## Due Friday July 23nd, 11:59pm


Please complete this notebook by filling in the cells provided. Before you begin, execute the following cell to load the provided tests. Each time you start your server, you will need to execute this cell again to load the tests.

In [1]:
# Don't change this cell; just run it. 
import numpy as np
import babypandas as bpd
import math
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('fivethirtyeight')

import otter
grader = otter.Notebook('tests')

### Deadline

This assignment is due Friday, July 23rd at 11:59pm. You are given six slip days thoughout the quarter which can extend the deadline by one day. See the syllabus for more details. With the exception of using slip days, late work will not be accepted unless you have made special arrangements with your instructor.

**Important**: The `otter` tests don't usually tell you that your answer is correct. More often, they help catch careless mistakes. It's up to you to ensure that your answer is correct. If you're not sure, ask someone (not for the answer, but for some guidance about your approach).

Directly sharing answers is not okay, but discussing problems with the course staff or with other students is encouraged. 

You should start early so that you have time to get help if you're stuck.

## 1. Ramen Ratings

![](data/menya.jpg)

In this section, we will be using a ramen rating dataset to better our understanding of A/B testing. The dataset can be found on [kaggle](https://www.kaggle.com/residentmario/ramen-ratings), but the data has been cleaned and condensed for the purposes of this question. We (the writers) also recommend eating at [Menya Ultra](http://menya-ultra.com/) before completing this section, as we did before we wrote these questions.

The ramen data is recorded in a CSV file called `ramen.csv`. It contains five columns: `Brand`, `Variety`, `Style`, `Region`, `Stars`. Read this file into a table called `ramen`.

In [2]:
ramen = bpd.read_csv('data/ramen.csv')
ramen

**Question 1.1**. You may have noticed that the `Stars` column contains strings instead of floats. Because we cannot do computations on strings, we need to convert these values into floats. In your `ramen` table, replace the `Stars` column so that all the data values are floats instead of strings. Find the mean star rating of all the ramen, and save it into a variable called `median_star`.

In [3]:
median_star = ...
median_star

In [ ]:
grader.check("q1_1")

**Question 1.2.** Notice that there are two styles of ramen: "Pack" and "Cup". Using `ramen`, calculate the difference between the median star ratings of Pack and Cup ramen. Assign your answer to `observed_difference`.

$$\text{observed difference} := \text{median Pack stars} - \text{median Cup stars}$$

In [6]:
observed_difference = ...
observed_difference

In [ ]:
grader.check("q1_2")

**Question 1.3.** Interpret in words the number you obtained for `observed_difference` and assign either 1, 2, 3, or 4 to `q1_3`.

1. In our sample, the median cup stars is lower than the median pack stars by about 25 percents.
2. In our sample, the median cup stars is higher than the median pack stars by about 25 percents.
3. In our sample, the median pack stars is lower than the median cup stars by about 0.25 stars.
4. In our sample, the median cup stars is lower than the median pack stars by about 0.25 stars.

In [9]:
q1_3 = ...

In [ ]:
grader.check("q1_3")

Now we want to conduct an A/B test (i.e. Permutation Test) to see if it is by chance that the **mean** star rating for the pack ramen is greater than cup ramen, or if the pack ramen really does have higher ratings than the cup. To remind you on the process of an A/B test, here is the textbook reference for the [process](https://eldridgejm.github.io/dive_into_data_science/07-hypothesis_testing/2_permutation_tests.html). In your upcoming A/B test, we want to shuffle the `Stars` column and keep the `Style` column in the same order.


**Null hypothesis:** Star ratings of pack ramen and cup ramen come from the same distribution.  
**Alternative hypothesis:** Star ratings of pack ramen is typically higher than that of cup ramen.

Hint: To make your simulation go faster, drop the irrelevant columns before our A/B test. Make another table called `small_ramen` that only has the `Stars` and `Style` column, and shuffle using `small_ramen`.

**Question 1.4.** Use a permutation test to calculate 100 differences using random permutations of the data. Store your 100 differences in the `differences` array.

In [12]:
differences = ...
for i in np.arange(100):
    differences = ...
differences

In [ ]:
grader.check("q1_4")

**Question 1.5.** Which of the follow choices best describes the purpose of the permutation test with regards to A/B testing? Assign either 1, 2, or 3 to `q1_5`.
1. The permutation test is a special case of the bootstrap and allows us to produce interval estimates.
2. The permutation test generates a null distribution which we can use in testing our hypothesis.
3. The permutation test mitigates noise in our data by generating new permutations of the data.

In [16]:
q1_5 = ...

In [ ]:
grader.check("q1_5")

**Question 1.6.** Compute a p-value for the hypothesis. That is, under the null hypothesis, compute the probability that we would have obtained a difference greater than or equal to `observed_difference` by chance alone. Assign your answer to `p_val`.

In [19]:
p_val = ...
p_val

In [ ]:
grader.check("q1_6")

**Question 1.7.** Do you reject or fail to reject the null hypothesis at the 0.05 significance level?

1. We fail to reject the null hypothesis at the 0.05 significance level.
2. We reject the null hypothesis at the 0.05 significance level.

In [22]:
q1_7 = ...

In [ ]:
grader.check("q1_7")

**Question 1.8.** Suppose in this question you had shuffled the `Style` column and kept the `Stars` column in the same order. 
Which of the following is a true statement?

1. Your new p-value would be 1 - (old p-value), where new p-value is with `Style` shuffled and old p-value is with `Stars` shuffled.
2. There would be no difference in the A/B Test if we had shuffled the `Style` column instead.
3. We would conclude that pack ramen would have lower star ratings than cup ramen.
4. The `Style` column cannot be shuffled because there are only two unique values.

In [25]:
q1_8 = ...

In [ ]:
grader.check("q1_8")

## 2. Percentiles

**The General Definition**

> Let $p$ be a number between 0 and 100. The $p$th percentile of a collection is the smallest value in the collection that is *at least as large* as $p$% of all the values. 

![](data/percentile_example.jpg)

By this definition, any percentile between 0 and 100 can be computed for any collection of values and is always an element of the collection. Suppose there are $n$ elements in the collection. To find the $p$th percentile:

1. Sort the collection in increasing order.
2. Find $p$% of $n$: $\frac p{100}*n$. Call that $h$. If $h$ is an integer, define $k = h$. Otherwise, let $k$ be the smallest integer greater than $h$.
3. Take the $k$th element of the sorted collection.

**Question 2.1.** Assign the number of elements in `values` to the variable `n`. Define `k` as above -- your answer should be an integer. Assign the 74th percentile of the array `values` to `seventy_fourth_percentile`. You must use the variables provided for you when solving this problem. For this problem only, you may *not* use `np.percentile()`.

*Hint:* Using `math.ceil()` will round up a number to the next nearest whole number. `math` has already been imported for you.

In [28]:
# don't change the values in this array!
values = np.array([23, 76, 94, 60, 71, 35, 23, 106, 54, 86, 39, 127, 112])
values.sort()  # This line sorts the array
values

In [29]:
seventy_fourth_percentile = ...
seventy_fourth_percentile

In [ ]:
grader.check("q2_1")

**Question 2.2.** The csv file `mcdonalds.csv` contains some selected information on menu items taken from [kaggle](https://www.kaggle.com/mcdonalds/nutrition-facts). The columns include `Category`, `Item`, `Calories`, `Sodium`, `Total Fat`, `Carbohydrates`, `Sugars`, `Protein`. Pick the appropriate bins to plot a histogram showing the distribution of `Calories`.

In [34]:
# do not change this cell
mcd = bpd.read_csv('data/mcdonalds.csv',index_col = 0)

1. mcd_bins = np.arange(0, 2000, 100)
2. mcd_bins = np.arange(0, 2000, 500)
3. mcd_bins = np.arange(0, 4000, 500)

In [35]:
mcd

In [36]:
q2_2 = ...
q2_2

In [ ]:
grader.check("q2_2")

Plot a histogram showing the distribution of `Calories` using the bins chosen from Question 2.2

In [39]:
mcd_bins = np.arange(0, 2000, 100)

**Question 2.3.** Compare the calorie distribution between categories `Beef & Pork` **AND** `Chicken & Fish` (group 1) **versus** category `Salads` (group 2). Find the absolute difference between the 90th percentile of the two group's `Calories` column and assign it to `absolute_difference`. You may use `np.percentile()`.

In [40]:
...
absolute_difference

In [ ]:
grader.check("q2_3")

**Question 2.4**. In an array `fat_quartiles`, put the values for the first, second, and third quartiles (in that order) of the `Total Fat` data provided, but only for items not in the `Breakfast` category. Make sure your values are in the correct order. You may use `np.percentile()`.

In [43]:
...
fat_quartiles = ...
fat_quartiles

In [ ]:
grader.check("q2_4")

**Quetion 2.5.** Say that McDonald's wants to add in a new Smoothie called `Mocha Almond Fudge (Medium)` which has 60 grams of sugar. What would the `Sugars` percentile range of this new smoothie be out of the **Medium** only `Smoothies & Shakes` `Category`? Give the result back as two numbers (1-100). The smallest percentile that will return the new drink should be returned as `lower_bound` and the largest percentile that will return the new drink should be labeled `upper_bound`. For example, if the new smoothie would be returned back when finding the 70th percentile and 80th percentile of the Medium Smoothies, but not at the 69th and 81st percentile, then lower_bound = 70, upper_bound = 80.

**Hint:** If you're unsure about percentiles, refer back to the general definition above Question 1.

In [46]:
lower_bound = ...
upper_bound = ...

In [ ]:
grader.check("q2_5")

**Question 2.6.** Shaun surveyed his class to find the total number of pets each of his classmates has. You can see his findings below in the table `pets`. For instance, 2 people have 0 pets, 4 have 1 pet, and so on. If one of his classmates, Jake, has some number of pets that falls in the 42nd percentile of Shaun's data, how many pets does Jake have? Assign your answer to the value `jake_pets`. You may use `np.percentile()`.

*Hint*: A possible solution uses [np.repeat](https://docs.scipy.org/doc/numpy/reference/generated/numpy.repeat.html). (Also described in part 4.3)

In [55]:
# load the data
pets = bpd.read_csv('data/pets.csv')
pets

In [56]:
...
jake_pets = ...
jake_pets # don't worry if your number appears to be a float with many digits

In [ ]:
grader.check("q2_6")

## 3. In-N-Out and Five Guys

Suppose you are a burger lover and a regular at Five Guys. When you get your twentieth Five Guys burger of the week, you notice that your patty is extremely small. Your friend tells you Five Guys patties have always been this small, but you are doubtful and decide to investigate.

Ideally, you would want to figure out the exact mean weight of all Five Guys burger patties. However, it's not feasible to obtain the mean weight of *all* Five Guys patties (i.e. the mean weight of the population).

**Question 3.1.** Complete the statement below by filling in the blanks.

<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">

Therefore, you want to collect a sample of Five Guys patties to obtain a ____________ statistic to estimate the ____________ parameter.

<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">

1. population; sample
2. sample; population

In [60]:
q3_1 = ...

In [ ]:
grader.check("q3_1")

Your other friend, who works at Five Guys, agreed to weigh all the patties during his shift. He also does the same with In-N-Out, since he works there as well. You decide to use this data as your sample.

**Question 3.2.** Your data is recorded in a CSV file called `burgers.csv`. Read this file into a table named `burgers`.

In [63]:
burgers = bpd.read_csv('data/burgers.csv')
burgers

In [ ]:
grader.check("q3_2")

**Question 3.3.** For now, you only care about the weights of the Five Guys patties. Create a new table with the rows of `burgers` where the value of `Place` is "Five Guys". Assign this new table to `five_guys`.

In [65]:
five_guys = ...
five_guys

In [ ]:
grader.check("q3_3")

**Question 3.4.** Calculate the mean weight of `five_guys` patties and assign it to `five_guys_mean`. 

In [69]:
five_guys_mean = ...
five_guys_mean

In [ ]:
grader.check("q3_4")

You're done! Or are you? You have a single point estimate for the true mean Five Guys patty weight. However, you don't know how uncertain your estimate is and you don't know how much these estimates could vary. In other words, you don't have a sense of how good your estimate is. You may have gotten a particular statistic for one sample, but you could also get a completely different one for another sample.

This is where the idea of resampling via the [bootstrap](https://eldridgejm.github.io/dive_into_data_science/08-estimation/1_bootstrap.html) comes in. Let's assume that our original sample resembles the population fairly well. We can then resample from our original sample to produce even more estimates, which we can then use to produce an interval estimate for the true mean weight of Five Guys patties.

**Question 3.5.** Fill out the following code to produce 2,000 bootstrapped estimates for the  *mean* weight of Five Guys patties. Store your 2,000 estimates in the `five_guys_means` array.

In [72]:
five_guys_means = ...
for i in np.arange(2000):
    resample = ...
    resample_mean = ...
    five_guys_means = ...

In [ ]:
grader.check("q3_5")

In [76]:
#: This cell displays a histogram of five_guys_means
bpd.DataFrame().assign(Estimated_Mean = five_guys_means).plot(kind = 'hist')

**Question 3.6.** Using the array `five_guys_means`, compute an approximate 90% confidence interval for the true mean weight of Five Guys patties. (Compute the lower and upper ends of the interval, named `lower_bound` and `upper_bound`, respectively.)

*Hint:* Use `percentile()`.

In [77]:
lower_bound = ...
upper_bound = ...

#: the confidence interval
print("Bootstrapped 90% confidence interval for the true mean weight of Five Guys patties: [{:f}, {:f}]".format(lower_bound, upper_bound))

In [ ]:
grader.check("q3_6")

**Question 3.7.** Which of the following would make the histogram narrower? Assign either 1 or 2 to `q3_7`.
1. Starting with a larger original sample size.
2. Increasing the number of resamples (repetitions of bootstrap).

In [83]:
q3_7 = ...
q3_7

In [ ]:
grader.check("q3_7")

**Question 3.8.** Suppose you want to find the weight of the lightest Five Guys patty (minimum weight out of the entire population). Would your bootstrap procedure be effective in estimating the weight of the lightest Five Guys patty?

1. Yes, a bootstrap procedure would be effective in estimating the weight of the lightest Five Guys patty.
2. No, a bootstrap procedure would not be effective in estimating the weight of the lightest Five Guys patty.

In [86]:
q3_8 = 2

In [ ]:
grader.check("q3_8")

**Question 3.9.** Suppose you're wondering how heavy the average In-N-Out patty is compared to the average Five Guys patty. Using the same bootstrap procedure, compute an approximate 90% confidence interval for the true mean difference in weight between In-N-Out and Five Guys patties. Store your 1,000 estimates in the `difference_means` array. Use the original `burgers` table for this.

$$\text{difference_mean} := \text{mean weight of In-N-Out} - \text{mean weight of Five Guys}$$

In [89]:
difference_means = ...
...
difference_means

In [ ]:
grader.check("q3_9")

In [94]:
# This cell displays a histogram of difference_means
bpd.DataFrame().assign(Estimated_Mean_Difference = difference_means).plot(kind = 'hist')

**Question 3.10.** Compute the 90% confidence interval for the mean difference in weights of In-N-Out and Five Guys patties. Assign the left and right endpoints to `left_endpoint` and `right_endpoint` respectively. 

In [95]:
left_endpoint = ...
right_endpoint = ...

print("Bootstrapped 95% confidence interval for the mean difference in weights of In-N-Out and Five Guys patties: [{:f}, {:f}]".format(left_endpoint, right_endpoint))

In [ ]:
grader.check("q3_10")

**Question 3.11:** Based on your histogram and confidence interval, would you say that the mean In-N-Out patty is likely to be lighter than the mean Five Guys patty?

1. No, it is not likely that the mean In-N-Out patty is lighter than the mean Five Guys patty.
2. Yes, it is likely that the mean In-N-Out patty is lighter than the mean Five Guys patty.

In [101]:
q3_11 = ...

In [ ]:
grader.check("q3_11")

**Question 3.12.** Would changing the units of weight from ounces to grams change your conclusion? Assign a boolean (`True` if it would and `False` otherwise) to the name `q3_12`.

In [104]:
q3_12 = ...

In [ ]:
grader.check("q3_12")

**Question 3.13.** You found that the device you used to measure the weight was not properly calibrated -- even when nothing was on the scale it did not display a weight of zero, but rather some small, positive amount. After you adjust it and remeasure all patties from both restaurants, they all **become lighter by the same amount**. Does this change your conclusion? Assign a boolean (`True` if it would and `False` if it would not) to the name `q3_13`.

In [131]:
q3_13 = ...

In [ ]:
grader.check("q3_13")

# 4. Conceptual Questions

**Question 4.1.** Choose the best test (hypothesis, permutation (A/B), bootstrap) to test the following: <br>
You claim that the proportion of students from Socal that likes In-N-Out over five guys is higher than the proportion of students from Norcal that likes In-N-Out.
1. hypothesis testing
2. bootstrap
3. permutation (A/B) testing


In [110]:
q4_1 = ...

In [ ]:
grader.check("q4_1")

**Question 4.2.** Which test statistic(s) could we use to test the claim from 4.1? Put all that are valid into a list.
1. the number of Norcal students that likes In-N-Out over the total number of students that likes In-N-Out
2. the difference between the proportion of Norcal students that likes In-N-Out and proportion of Socal students that likes In-N-Out
3. total variation distance between the proportion of Norcal students that likes In-N-Out from Norcal and proportion of Socal students that likes In-N-Out


In [113]:
q4_2 = ...

In [ ]:
grader.check("q4_2")

**Question 4.3.** Choose the best test (hypothesis, permutation (A/B), bootstrap) to test the following: <br>
You claim that the proportion of students that prefer Norcal burgers over Socal burgers is less than the proportion of students that prefer Socal burgers.
1. hypothesis testing
2. permutation (A/B) testing
3. bootstrap

In [117]:
q4_3 = ...

In [ ]:
grader.check("q4_3")

**Question 4.4.** Which test statistic(s) could we use to test the claim from 4.3? Put all that are valid into a list.
1. the number of students that prefer Norcal burgers over total number of students
2. the difference between the proportion of students that prefer Norcal burgers and proportion of students that prefer Socal burgers
3. total variation distance between the proportion of students that prefer Norcal burgers and proportion of students that prefer Socal burgers

In [120]:
q4_4 = ...

In [ ]:
grader.check("q4_4")

**Question 4.5.** Choose the best test (hypothesis, permutation (A/B), bootstrap) to test the following:
You want to find the true median number of weekly burgers consumed by a student.
1. hypothesis testing
2. permutation (A/B) testing
3. bootstrap

In [124]:
q4_5 = ...

In [ ]:
grader.check("q4_5")

**Question 4.6.** After peforming bootstrapping, we construct several 95% confidence intervals. Which of the following is true about the confidence intervals?

1. The true parameter has a roughly 95% chance of falling in a confidence interval.
2. Roughly 95% of the population lie within the confidence intervals.
3. Roughly 95% of the confidence intervals constructed capture the true parameter. 

In [127]:
q4_6 = ...

In [ ]:
grader.check("q4_6")

**Question 4.7.** (Optional/Not graded) Why do we use *replacement* in bootstrapping and not in A/B testing? What’s the difference between the two?

<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">
Write your answer here.

<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">

# Finish Line

Congratulations! You are done with homework 6.

To submit your assignment:

1. Select `Kernel -> Restart & Run All` to ensure that you have executed all cells, including the test cells.
2. Read through the notebook to make sure everything is fine and all tests passed.
3. Run the cell below to run all tests, and make sure that they all pass.
4. Download your notebook using `File -> Download as -> Notebook (.ipynb)`, then upload your notebook to Gradescope.

In [130]:
# For your convenience, you can run this cell to run all the tests at once!
grader.check_all()